<a href="https://colab.research.google.com/github/ReyFrank92/joyofcoding/blob/main/Clasificaci%C3%B3n_petroleo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, GlobalMaxPool1D
from tensorflow.keras.layers import LSTM, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error

np.random.seed(0)
tf.random.set_seed(0)

In [19]:
from google.colab import files
data_to_load = files.upload()

Saving Resumen_petroleo.csv to Resumen_petroleo (1).csv


In [20]:
import io
df = pd.read_csv(io.BytesIO(data_to_load['Resumen_petroleo.csv']), index_col='Fecha', parse_dates =True)

In [21]:
# N = number of samples
# T = sequence length
# D = number of input features
# M = number of hideen units 
# K = number of output units

In [22]:
df['Log_MM'] = np.log(df['MM'])

In [23]:
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
np.random.seed(0)
tf.random.set_seed(0)

In [25]:
input_data = df[['WTI', 'BRENT','DJI','XAU','Oil C1']].values
targets = df['Log_MM'].values


In [26]:
print(input_data.shape)

(318, 5)


In [27]:
print(targets.shape)

(318,)


In [28]:
# Now make the actual data which will go into the neural network
T = 10 #the number of time steps to look at to make a prediction for the next day
D = input_data.shape[1] #en este ejemplo son 5 porque usamos 5 variables disitntas
N = len(input_data) - T # (e.g. if T = 10 and you have 11 data points then you'd onlye have 1 sample )

In [29]:
# normalize the inputs
Ntrain = len(input_data) * 2 //3
scaler = StandardScaler()
scaler.fit(input_data[:Ntrain + T]) #escala los primeros 2/3 de datos + T
input_data = scaler.transform(input_data) # remplaza el dataset a los datos escalados

In [30]:
# Setup X_train and Y_train
X_train = np.zeros((Ntrain, T, D)) #crea ceros del tensor de 3 dimensiones y la medida son los números de cada definición (1249,5,10)
Y_train = np.zeros(Ntrain)

for t in range(Ntrain): 
  X_train[t, :, :] = input_data[t:t+T] #es el recorrido que hacen los datos de 1 en 1 con las 5 variables
  Y_train[t] = (targets[t+T]>0) #booleano, mientras t+T sean > a 0 entonces Y_train será igual a x numero

In [33]:
# Setup X_test and Y_test
X_test = np.zeros((N - Ntrain, T, D)) #crea ceros del tensor de 3 dimensiones y la medida son los números de cada definición (1249-2/3 Ntrain ,5,10)
Y_test = np.zeros(N - Ntrain) # (1249-839)

#Acá llenamos los ceros generados
for u in range(N - Ntrain): 
  # u ocunts from 0... (N - Ntrain)
  # t counts form Ntrain...N 
  t = u + Ntrain #(U toma de 0 hasta N, Ntrain siempre es 839)
  X_test[u, :, :] = input_data[t:t+T] #es el recorrido que hacen los datos de 1 en 1 con las 5 variables
  Y_test[u] = (targets[t+T]>0) #booleano, mientras t+T sean > a 0 entonces Y_train será igual a x numero

In [34]:
print(X_train.shape)

(212, 10, 5)


In [35]:
print(X_test.shape)

(96, 10, 5)


In [38]:
212+96+10

318

In [44]:
# Make the RNN []
i = Input(shape=(T,D))  #es solo poner una entrada, en este caso 10 porque es la primera capa y uno busca un AR(10) #Aca si se pone (T,1)
x = LSTM(5)(i) #50 = number of hidden units // #i = (10,5)(especificación de la entrada)
x = Dense(1, activation='sigmoid')(x)  # final layer is one #solo es una capa. 
model = Model(i, x)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate= 0.001),
    metrics = ['accuracy'],
)

In [45]:
# train the RNN
r = model.fit(
    X_train, Y_train,
    batch_size=32,
    epochs=300,
    validation_data=(X_test, Y_test),
)

Epoch 1/300
7/7 [==============================] - 5s 101ms/step - loss: nan - accuracy: 0.0755 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/300
7/7 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/300
7/7 [==============================] - 0s 13ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/300
7/7 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/300
7/7 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/300
7/7 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/300
7/7 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 8/3